# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [12]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import geopandas as gpd

# Import API key
from api_keys import geoapify_key

In [9]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")
# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,edinburgh of the seven seas,-37.0676,-12.3116,291.59,98,100,11.06,SH,1704782899
1,1,arawa,-6.2132,155.5367,301.29,88,100,1.13,PG,1704782900
2,2,chibuto,-24.6867,33.5306,303.25,50,7,1.75,MZ,1704782900
3,3,hermanus,-34.4187,19.2345,293.69,69,10,5.00,ZA,1704782895
4,4,pyt-yakh,60.7499,72.8582,251.85,92,76,4.77,RU,1704782900


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [13]:
%%capture --no-display
gdf = gpd.GeoDataFrame(city_data_df, geometry=gpd.points_from_xy(city_data_df['Lng'], city_data_df['Lat']))

# Configure the map plot
# YOUR CODE HERE
humidity_map = gdf.hvplot.points(
    geo=True,
    x='Lng',
    y='Lat',
    size='Humidity',
    cmap='viridis',
    colorbar=True,
    title='City Humidity Map',
    tools=['hover']
).opts(width=800, height=500)

# Display the map
# YOUR CODE HERE
humidity_map

:Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [22]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
ideal_conditions_df = city_data_df[
    (city_data_df['Max Temp'] > 21) & (city_data_df['Max Temp'] < 27) &
    (city_data_df['Wind Speed'] < 4.5) & (city_data_df['Cloudiness'] == 0)
]

# Drop any rows with null values
# YOUR CODE HERE
ideal_conditions_df = ideal_conditions_df.dropna()

# Display sample data
# YOUR CODE HERE
ideal_conditions_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date


### Step 3: Create a new DataFrame called `hotel_df`.

In [23]:
# Use the Pandas copy function to create DataFrame called hotel_df
hotel_df = ideal_conditions_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame
hotel_df = hotel_df.assign(Hotel_Name="")

# Display sample data
hotel_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel_Name


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [24]:
# Set parameters to search for a hotel
radius = 10000  # 10,000 meters radius
params = {
    "apiKey": "0eb85704e8824d3197e46360ecc9a241",  
    "radius": radius,
    "filter": "amenity:hotel",
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"amenity:hotel"
    params["bias"] = f"circle:{radius}@{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()


Starting hotel search


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel_Name


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [25]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
hotel_map = gdf.hvplot.points(
    geo=True,
    x='Lng',
    y='Lat',
    size='Humidity',
    cmap='viridis',
    colorbar=True,
    title='City Humidity Map',
    tools=['hover']
).opts(width=800, height=500, hover_cols=['City', 'Country', 'Humidity', 'Hotel Name'])

# Display the map
hotel_map


ValueError: Unexpected option 'hover_cols' for Points type across all extensions. Similar options for current extension ('bokeh') are: ['hover_color', 'hover_fill_color', 'hover_line_color'].